# Interactive Learning Course
## HW #2, Multi-Armed Bandits

In [216]:
import numpy as np
from abc import ABC, abstractmethod
import gym
import matplotlib.pyplot as plt

# Part 2

## Reward class

In [ ]:
class Reward(ABC):
    def __init__(self, debts=[5, 20, 100]):
        super(Reward, self).__init__()
        self.debts = debts
        
    @abstractmethod
    def get_reward(self, action):
        pass

In [ ]:
class Student(Reward):
    def __init__(self):
        super(Student, self).__init__()

    def get_reward(self, action):
        if action == 0:
            obs = np.random.choice([4, 5, 5.1], p=[0.2, 0.2, 0.6]) - 5.0
        elif action == 1:
            obs = np.random.choice([5, 10, 20, 20.75], p=[0.5, 0.3, 0.1, 0.1]) - 20.0
        else:
            obs = np.random.choice([10, 20, 50, 100, 105], p=[0.5, 0.3, 0.1, 0.05, 0.05]) - 100.0
        
        return obs, obs - self.debts[action]

In [ ]:
class GovStaff(Reward):
    def __init__(self):
        super(GovStaff, self).__init__()

    def get_reward(self, action):
        if action == 0:
            obs = np.random.choice([4, 5, 5.1], p=[0.05, 0.3, 0.65])
        elif action == 1:
            obs = np.random.choice([5, 10, 20, 20.75], p=[0.1, 0.1, 0.3, 0.5])
        else:
            obs = np.random.choice([20, 50, 80, 100, 105], p=[0.1, 0.1, 0.2, 0.3, 0.3])
        
        return obs, obs - self.debts[action]

In [ ]:
class SelfEmp(Reward):
    def __init__(self):
        super(SelfEmp, self).__init__()

    def get_reward(self, action):
        if action == 0:
            obs = np.random.choice([0, 2.5, 5.1], p=[0.8, 0.1, 0.1])
        elif action == 1:
            obs = np.random.choice([5, 10, 20, 20.75], p=[0.05, 0.05, 0.3, 0.6])
        else:
            obs = np.random.choice([20, 50, 80, 100, 105], p=[0.02, 0.03, 0.05, 0.2, 0.7])
        
        return obs, obs - self.debts[action]

## Environment

In [ ]:
class Environment:
    def __init__(self, id, reward):
        ## YOUR CODES ##
    
    def calc_reward(self, action):
        ## YOUR CODES ##

    def get_available_actions(self):
        ## YOUR CODES ##

# Part 3

In [241]:
class Agent:
    def __init__(self, _id, env=None, alpha=0, beta=1, gamma=1):
        ## YOUR CODES ##

    def choose_action(self):
        ## YOUR CODES ##
    
    def calculate_utility(self, reward):
        ## YOUR CODES ##
    
    def take_action(self, action):
        ## YOUR CODES ##
    
    def update_value(self, action, util):
        ## YOUR CODES ##

    def step(self, best_reward, trial):
        ## YOUR CODES ##

    def reset(self):
        ## YOUR CODES ##
    
    def get_rewards(self):
        ## YOUR CODES ##
    
    def get_regrets(self):
        ## YOUR CODES ##